# FastText工具实现
## FastText优势

在保持较高精度的情况下, 快速的进行训练和预测
    
    fasttext工具包中内含的fasttext模型具有十分简单的网络结构
    使用fasttext模型训练词向量时使用层次softmax结构, 来提升超多类别下的模型性能
    由于fasttext模型过于简单无法捕捉词序特征, 因此会进行n-gram特征提取以弥补模型缺陷提升精度


In [ ]:
# 安装
git clone https://github.com/facebookresearch/fastText.git
cd fastText
# 使用pip安装python中的fasttext工具包
sudo pip install .

## 文本分类

+ 二分类：
    - 文本被分类两个类别中, 往往这两个类别是对立面, 比如: 判断一句评论是好评还是差评
    
+ 单标签多分类：
    - 文本被分入到多个类别中, 且每条文本只能属于某一个类别(即被打上某一个标签), 比如: 输入一个人名, 判断它是来自哪个国家的人名
    
+ 多标签多分类：
    - 文本被分人到多个类别中, 但每条文本可以属于多个类别(即被打上多个标签), 比如: 输入一段描述, 判断可能是和哪些兴趣爱好有关, 一段描述中可能即讨论了美食, 又太讨论了游戏爱好


## 使用fasttext工具进行文本分类的过程

1. 获取数据

2. 训练集与验证集的划分

3. 训练模型

4. 使用模型进行预测并评估

5. 模型调优

6. 模型保存与重加载

## 获取数据

facebook AI实验室提供的演示数据集

cooking.stackexchange.txt中的每一行都包含一个标签列表，后跟相应的文档, 标签列表以类似"__label__sauce __label__cheese"的形式展现, 代表有两个标签sauce和cheese, 所有标签__label__均以前缀开头，这是fastText识别标签或单词的方式. 标签之后的一段话就是文本信息.如: How much does potato starch affect a cheese sauce recipe?


In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt    

--2020-07-06 21:08:35--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
正在解析主机 dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142
正在连接 dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：457609 (447K) [application/x-tar]
正在保存至: “cooking.stackexchange.tar.gz.1”

cooking.stackexchan 100%[===================>] 446.88K  1.04MB/s  用时 0.4s      

2020-07-06 21:08:37 (1.04 MB/s) - 已保存 “cooking.stackexchange.tar.gz.1” [457609/457609])

x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt
__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skil

## 数据集划分

In [3]:
# 查看数据集
!wc cooking.stackexchange.txt

   15404  169582 1401900 cooking.stackexchange.txt


In [4]:
# 12404条数据作为训练数据
! head -n 12404 cooking.stackexchange.txt > cooking.train
# 3000条数据作为验证数据
! tail -n 3000 cooking.stackexchange.txt > cooking.valid
! ls 

FastText文本分类.ipynb         cooking.train
cooking.stackexchange.id       cooking.valid
cooking.stackexchange.tar.gz   readme.md
cooking.stackexchange.tar.gz.1 readme.txt
cooking.stackexchange.txt


## 模型训练

In [8]:
import fasttext
# 使用fasttext的train_supervised方法进行文本分类模型的训练
model = fasttext.train_supervised(input="cooking.train")  

In [10]:
# 通过我们常识可知预测是错误的
model.predict("Why not put knives in the dishwasher?")

(('__label__food-safety',), array([0.06222661]))

In [11]:
# 为了评估模型到底表现如何, 我们在3000条的验证集上进行测试
model.test("cooking.valid")
# 元组中的每项分别代表, 验证集样本数量, 精度以及召回率 
# 我们看到模型精度和召回率表现都很差, 接下来我们讲学习如何进行优化.


(3000, 0.15266666666666667, 0.06602277641631829)

## 模型预测与评估


In [9]:
# 使用模型预测一段输入文本, 通过我们常识, 可知预测是正确的, 但是对应预测概率并不大
model.predict("a")

(('__label__baking',), array([0.05981867]))

## 模型调优

In [14]:
# 数据预处理
# 规范化  标点符号与单词相连以及大小写不统一
! cat cooking.stackexchange.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > cooking.preprocessed.txt
! head -n 12404 cooking.preprocessed.txt > cooking.train
! tail -n 3000 cooking.preprocessed.txt > cooking.valid

In [16]:
! head cooking.train

__label__sauce __label__cheese how much does potato starch affect a cheese sauce recipe ? 
__label__food-safety __label__acidity dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove how do i cover up the white spots on my cast iron stove ? 
__label__restaurant michelin three star restaurant; but if the chef is not there
__label__knife-skills __label__dicing without knife skills ,  how can i quickly and accurately dice vegetables ? 
__label__storage-method __label__equipment __label__bread what ' s the purpose of a bread box ? 
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home ? 
__label__chocolate american equivalent for british chocolate terms
__label__baking __label__oven __label__convection fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise regulation and balancing of readymade packed mayonnaise and other sauces


In [17]:
# 重新训练
model = fasttext.train_supervised(input="cooking.train")

In [18]:
model.test("cooking.valid")

(3000, 0.17133333333333334, 0.07409543030128297)

# Pytorch实现